# Restaurant Topicnizer

In [2]:
import cluster
import os

import pandas as pd

from ipywidgets import Layout
from ipywidgets import HBox
from ipywidgets import Dropdown

NUM_TOPICS = 5

In [4]:
directory = os.fsencode("RestaurantData")
restaurant_files = []

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".csv"):
        restaurant_files.append(os.path.join(os.fsdecode(directory), filename))
        
restaurant_dfs = [pd.read_csv(x) for x in restaurant_files]
restaurant_dfs = [cluster.weighted_averages_by_year(x, NUM_TOPICS) for x in restaurant_dfs]
restaurant_names = [os.path.splitext(filename)[0] for filename in restaurant_files]
restaurant_names = [filename.replace('_', ' ')[filename.index("/")+1:] for filename in restaurant_names]
restaurant_names = [" ".join(filename.split()[:-2]) for filename in restaurant_names]

In [5]:
select_topic = Dropdown(options = ["Topic{}".format(i) for i in range(NUM_TOPICS)], layout=Layout(width = '160px'))
select_restaurant = Dropdown(options = restaurant_names, layout=Layout(width = '160px'))
HBox([select_restaurant, select_topic])